# Map2alm

Run the first cell to install requirements, then comment it out and do a "Restart and run all"

In [ ]:
#%pip install numpy==1.26.4 scipy==1.13.1 pixell==0.26.0 git+https://github.com/galsci/pysm.git

In [ ]:
import pysm3
from matplotlib import pyplot as plt
import numpy as np
import pixell
from pysm3 import units as u
import healpy as hp

## Define the resolution

In [ ]:
nside = 256  # higher for high resolution
healpix_reso = hp.nside2resol(nside) * u.radian
healpix_reso.to(u.arcmin)

In [ ]:
npix = hp.nside2npix(nside)
print(npix / 1e6, "Mpix")

## PySM simulation

In [ ]:
seed = 101

In [ ]:
dust = pysm3.ModifiedBlackBodyRealization(
    nside=nside,
    amplitude_modulation_temp_alm="dust_gnilc/raw/gnilc_dust_temperature_modulation_alms_lmax768.fits.gz",
    amplitude_modulation_pol_alm="dust_gnilc/raw/gnilc_dust_polarization_modulation_alms_lmax768.fits.gz",
    largescale_alm="dust_gnilc/raw/gnilc_dust_largescale_template_logpoltens_alm_nside2048_lmax1024_complex64.fits.gz",
    small_scale_cl="dust_gnilc/raw/gnilc_dust_small_scales_logpoltens_cl_lmax16384.fits.gz",
    largescale_alm_mbb_index="dust_gnilc/raw/gnilc_dust_largescale_template_beta_alm_nside2048_lmax1024.fits.gz",
    small_scale_cl_mbb_index="dust_gnilc/raw/gnilc_dust_small_scales_beta_cl_lmax16384_2023.06.06.fits.gz",
    largescale_alm_mbb_temperature="dust_gnilc/raw/gnilc_dust_largescale_template_Td_alm_nside2048_lmax1024.fits.gz",
    small_scale_cl_mbb_temperature="dust_gnilc/raw/gnilc_dust_small_scales_Td_cl_lmax16384_2023.06.06.fits.gz",
    freq_ref="353 GHz",
    seeds=[seed, seed + 1000, seed + 2000],
    max_nside=8192,
)

In [ ]:
sky = pysm3.Sky(nside=nside, component_objects=[dust])

In [ ]:
freq = 353 * u.GHz

In [ ]:
m = sky.get_emission(freq)

In [ ]:
hp.mollview(m[0], max=1e3)

### alm and cl calculation

In [ ]:
alm=hp.map2alm(m[0])
cl=pixell.curvedsky.alm2cl(alm)

In [ ]:
plt.plot(cl)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('l')
plt.ylabel('Spectrum')
plt.title(r'$c_l$ spectrum')

## Convert sky map to 2D

In [ ]:
car_reso = (np.pi / np.round(np.pi / healpix_reso.value)) * u.radian
print(healpix_reso.to(u.arcmin), car_reso.to(u.arcmin))

In [ ]:
pixell.__version__

In [ ]:
m_car = pysm3.apply_smoothing_and_coord_transform(
    m, output_car_resol=car_reso, return_healpix=False, return_car=True
)

In [ ]:
pysm3.__file__

In [ ]:
m_car.shape

In [ ]:
plt.imshow(m_car[0,:,:], vmin=0, vmax=700, origin="lower")

## Calculate the alm and cl from the 2D map

In [ ]:
lmax=3*nside-1
print(lmax)

In [ ]:
alm = pixell.curvedsky.map2alm(m_car,lmax=lmax)

In [ ]:
nside

In [ ]:
hp.mollview(hp.alm2map(alm, nside=nside)[0], max=1e3)

## Crop image to a a square centered in the middle

In [ ]:
img_size=512
start_x=int(len(m_car[0,0,:])/2-img_size/2)#500
start_y=int(len(m_car[0,:,0])/2-img_size/2)#
print('The image starts at x',start_x,'and y ',start_y)

In [ ]:
crop_image=m_car[0,start_y:start_y+img_size,start_x:start_x+img_size]

In [ ]:
plt.imshow(crop_image, vmin=0, vmax=700, origin="lower")

### Save image to file

In [ ]:
crop_image.shape

In [ ]:
fig = plt.figure(frameon=False)
plt.imshow(crop_image, vmin=0, vmax=1000, origin="lower")
plt.axis('off')
#fig.savefig('out.png', bbox_inches='tight', pad_inches=0)
plt.imsave(fname='out.png', arr=crop_image, vmin=0, vmax=1000, origin="lower", format='png') #imsave save the array "as is"

In [ ]:
lmax=3*nside-1
print(lmax)